In [ ]:
# Lasso,ridge

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


# 1. Read dataset and display columns

In [3]:
dataset=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Machine Learning-Regression/1.Traditional Way/6.Regularaztion_Lasso_Ridge_elastic/Lasso_Ridge_Elastic/cruise_ship_info.csv")

In [4]:
dataset.head()

Ship_name Cruise_line  Age  Tonnage  passengers  length  cabins  \
0      Journey     Azamara    6   30.277        6.94    5.94    3.55   
1        Quest     Azamara    6   30.277        6.94    5.94    3.55   
2  Celebration    Carnival   26   47.262       14.86    7.22    7.43   
3     Conquest    Carnival   11  110.000       29.74    9.53   14.88   
4      Destiny    Carnival   17  101.353       26.42    8.92   13.21   

   passenger_density   crew  
0              42.64   3.55  
1              42.64   3.55  
2              31.80   6.70  
3              36.99  19.10  
4              38.36  10.00

In [5]:
print("The Number of Row:",dataset.shape[0])
print("The Number of Column",dataset.shape[1])

The Number of Row: 158
The Number of Column 9


In [6]:
def segreQuanQual(dataset):
    quantative=[]
    qualtative=[]

    for i in dataset.columns:
    #print(i)
        if(dataset[i].dtypes =='object'):
            qualtative.append(i)
        else:
            quantative.append(i)
    print("The Quantitative Data:",quantative)
    print("The Qualtitative Data",qualtative)
    return quantative,qualtative

In [7]:
quantative,qualtative=segreQuanQual(dataset)

The Quantitative Data: ['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density', 'crew']
The Qualtitative Data ['Ship_name', 'Cruise_line']


In [8]:
#Descriptive Porcess
def descriptive_Analysis(dataset,quantative):
    des_data=pd.DataFrame(index=["Count","Mean","Median","Mode","Std","Min","Q1:25%","Q2:50%","Q3:75%","Q4:100%","IQR","1.5Rule",
                            "Lesser","Greater"],columns=quantative)

    for i in quantative:
        des_data[i]["Count"]=len(dataset[i])
        des_data[i]["Mean"]=dataset[i].mean()
        des_data[i]["Median"]=dataset[i].median()
        des_data[i]["Mode"]=dataset[i].mode()[0]
        des_data[i]["Std"]=dataset[i].describe()["std"]
        des_data[i]["Min"]=dataset[i].describe()["min"]
        des_data[i]["Q1:25%"]=dataset[i].describe()["25%"]
        des_data[i]["Q2:50%"]=dataset[i].describe()["50%"]
        des_data[i]["Q3:75%"]=dataset[i].describe()["75%"]
        des_data[i]["Q4:100%"]=dataset[i].describe()["max"]
        des_data[i]["IQR"]=des_data[i]["Q3:75%"]-des_data[i]["Q1:25%"]
        des_data[i]["1.5Rule"]=1.5* des_data[i]["IQR"]
        des_data[i]["Lesser"]= des_data[i]["Q1:25%"]-des_data[i]["1.5Rule"]
        des_data[i]["Greater"]= des_data[i]["Q3:75%"]+des_data[i]["1.5Rule"]

    return des_data

In [9]:
des_Data=descriptive_Analysis(dataset,quantative)
des_Data

Age    Tonnage passengers    length    cabins  \
Count          158        158        158       158       158   
Mean     15.689873  71.284671  18.457405  8.130633      8.83   
Median        14.0     71.899       19.5     8.555      9.57   
Mode            12     70.367      20.52      9.51      9.75   
Std       7.615691   37.22954   9.677095  1.793474  4.471417   
Min            4.0      2.329       0.66      2.79      0.33   
Q1:25%        10.0     46.013     12.535       7.1    6.1325   
Q2:50%        14.0     71.899       19.5     8.555      9.57   
Q3:75%        20.0    90.7725     24.845      9.51    10.885   
Q4:100%       48.0      220.0       54.0     11.82      27.0   
IQR           10.0    44.7595      12.31      2.41    4.7525   
1.5Rule       15.0   67.13925     18.465     3.615   7.12875   
Lesser        -5.0  -21.12625      -5.93     3.485  -0.99625   
Greater       35.0  157.91175      43.31    13.125  18.01375   

        passenger_density      crew  
Count                 158       158  
Mean            39.900949  7.794177  
Median             39.085      8.15  
Mode                34.29       9.2  
Std              8.639217  3.503487  
Min                  17.7      0.59  
Q1:25%              34.57      5.48  
Q2:50%             39.085      8.15  
Q3:75%             44.185      9.99  
Q4:100%             71.43      21.0  
IQR                 9.615      4.51  
1.5Rule           14.4225     6.765  
Lesser            20.1475    -1.285  
Greater           58.6075    16.755

In [10]:
def outliercolumn(quantative,des_data):
    lesser=[]
    greater=[]

    for i in quantative:
        if(des_data[i]["Lesser"]>des_data[i]['Min']):
            lesser.append(i)
        if(des_data[i]['Greater']<des_data[i]['Q4:100%']):
            greater.append(i)

    print("Lesser Range",lesser)
    print("Greater Range",greater)
    return lesser,greater

In [11]:
lesser,greater=outliercolumn(quantative,des_Data)

Lesser Range ['length', 'passenger_density']
Greater Range ['Age', 'Tonnage', 'passengers', 'cabins', 'passenger_density', 'crew']


In [12]:
# if outlier present run this
def changeoutlier(dataset,des_Data):
    for i in lesser:
        dataset[i][dataset[i]<des_Data[i]['Lesser']]=des_Data[i]['Lesser']
    #print(dataset[i])
    for j in greater:
        dataset[j][dataset[j]>des_Data[j]['Greater']]=des_Data[j]['Greater']
    return des_Data

In [13]:
change_Data=changeoutlier(dataset,des_Data)
change_Data

<ipython-input-12-5bab3f1b56fd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[i][dataset[i]<des_Data[i]['Lesser']]=des_Data[i]['Lesser']
<ipython-input-12-5bab3f1b56fd>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[j][dataset[j]>des_Data[j]['Greater']]=des_Data[j]['Greater']


Age    Tonnage passengers    length    cabins  \
Count          158        158        158       158       158   
Mean     15.689873  71.284671  18.457405  8.130633      8.83   
Median        14.0     71.899       19.5     8.555      9.57   
Mode            12     70.367      20.52      9.51      9.75   
Std       7.615691   37.22954   9.677095  1.793474  4.471417   
Min            4.0      2.329       0.66      2.79      0.33   
Q1:25%        10.0     46.013     12.535       7.1    6.1325   
Q2:50%        14.0     71.899       19.5     8.555      9.57   
Q3:75%        20.0    90.7725     24.845      9.51    10.885   
Q4:100%       48.0      220.0       54.0     11.82      27.0   
IQR           10.0    44.7595      12.31      2.41    4.7525   
1.5Rule       15.0   67.13925     18.465     3.615   7.12875   
Lesser        -5.0  -21.12625      -5.93     3.485  -0.99625   
Greater       35.0  157.91175      43.31    13.125  18.01375   

        passenger_density      crew  
Count                 158       158  
Mean            39.900949  7.794177  
Median             39.085      8.15  
Mode                34.29       9.2  
Std              8.639217  3.503487  
Min                  17.7      0.59  
Q1:25%              34.57      5.48  
Q2:50%             39.085      8.15  
Q3:75%             44.185      9.99  
Q4:100%             71.43      21.0  
IQR                 9.615      4.51  
1.5Rule           14.4225     6.765  
Lesser            20.1475    -1.285  
Greater           58.6075    16.755

In [14]:
des_Data=descriptive_Analysis(dataset,quantative)
des_Data

Age    Tonnage passengers    length    cabins  \
Count          158        158        158       158       158   
Mean     15.512658  70.877373   18.38481   8.14269  8.772136   
Median        14.0     71.899       19.5     8.555      9.57   
Mode            12     70.367      20.52      9.51      9.75   
Std       7.022926  35.920392   9.448795  1.759501  4.290235   
Min            4.0      2.329       0.66     3.485      0.33   
Q1:25%        10.0     46.013     12.535       7.1    6.1325   
Q2:50%        14.0     71.899       19.5     8.555      9.57   
Q3:75%        20.0    90.7725     24.845      9.51    10.885   
Q4:100%       35.0  157.91175      43.31     11.82  18.01375   
IQR           10.0    44.7595      12.31      2.41    4.7525   
1.5Rule       15.0   67.13925     18.465     3.615   7.12875   
Lesser        -5.0  -21.12625      -5.93     3.485  -0.99625   
Greater       35.0  157.91175      43.31    13.125  18.01375   

        passenger_density      crew  
Count                 158       158  
Mean            39.618481  7.752468  
Median             39.085      8.15  
Mode                34.29       9.2  
Std              7.757618  3.372034  
Min               20.1475      0.59  
Q1:25%              34.57      5.48  
Q2:50%             39.085      8.15  
Q3:75%             44.185      9.99  
Q4:100%           58.6075    16.755  
IQR                 9.615      4.51  
1.5Rule           14.4225     6.765  
Lesser            20.1475    -1.285  
Greater           58.6075    16.755

In [15]:
lesser,greater=outliercolumn(quantative,des_Data)

Lesser Range []
Greater Range []


In [16]:
cols_selected = ['Tonnage', 'passengers', 'length', 'cabins','crew']

In [17]:
dataset[cols_selected].head()

Tonnage  passengers  length  cabins    crew
0   30.277        6.94    5.94    3.55   3.550
1   30.277        6.94    5.94    3.55   3.550
2   47.262       14.86    7.22    7.43   6.700
3  110.000       29.74    9.53   14.88  16.755
4  101.353       26.42    8.92   13.21  10.000

In [18]:
df=dataset

In [19]:
X = df[cols_selected].iloc[:,0:4].values    # features matrix
y = df[cols_selected]['crew'].values        # target variable

In [20]:
X.shape

(158, 4)

In [21]:
y.shape

(158,)

#  One-hot encoding for categorical features

In [22]:
df2=pd.get_dummies(dataset[['Ship_name', 'Cruise_line','Age', 'Tonnage', 'passengers', 'length', 'cabins','passenger_density','crew']],drop_first=True)

In [23]:
df2.head()

Age  Tonnage  passengers  length  cabins  passenger_density    crew  \
0    6   30.277        6.94    5.94    3.55              42.64   3.550   
1    6   30.277        6.94    5.94    3.55              42.64   3.550   
2   26   47.262       14.86    7.22    7.43              31.80   6.700   
3   11  110.000       29.74    9.53   14.88              36.99  16.755   
4   17  101.353       26.42    8.92   13.21              38.36  10.000   

   Ship_name_Allegra  Ship_name_Amsterdam  Ship_name_Arcadia  ...  \
0                  0                    0                  0  ...   
1                  0                    0                  0  ...   
2                  0                    0                  0  ...   
3                  0                    0                  0  ...   
4                  0                    0                  0  ...   

   Cruise_line_Oceania  Cruise_line_Orient  Cruise_line_P&O  \
0                    0                   0                0   
1                    0                   0                0   
2                    0                   0                0   
3                    0                   0                0   
4                    0                   0                0   

   Cruise_line_Princess  Cruise_line_Regent_Seven_Seas  \
0                     0                              0   
1                     0                              0   
2                     0                              0   
3                     0                              0   
4                     0                              0   

   Cruise_line_Royal_Caribbean  Cruise_line_Seabourn  Cruise_line_Silversea  \
0                            0                     0                      0   
1                            0                     0                      0   
2                            0                     0                      0   
3                            0                     0                      0   
4                            0                     0                      0   

   Cruise_line_Star  Cruise_line_Windstar  
0                 0                     0  
1                 0                     0  
2                 0                     0  
3                 0                     0  
4                 0                     0  

[5 rows x 163 columns]

In [24]:
#Data partitioning into training and testing sets

In [25]:
from sklearn.model_selection import train_test_split
X = dataset[cols_selected].iloc[:,0:4].values
y = dataset[cols_selected]['crew']

In [26]:
X

array([[ 30.277  ,   6.94   ,   5.94   ,   3.55   ],
       [ 30.277  ,   6.94   ,   5.94   ,   3.55   ],
       [ 47.262  ,  14.86   ,   7.22   ,   7.43   ],
       [110.     ,  29.74   ,   9.53   ,  14.88   ],
       [101.353  ,  26.42   ,   8.92   ,  13.21   ],
       [ 70.367  ,  20.52   ,   8.55   ,  10.2    ],
       [ 70.367  ,  20.52   ,   8.55   ,  10.2    ],
       [ 70.367  ,  20.56   ,   8.55   ,  10.22   ],
       [ 70.367  ,  20.52   ,   8.55   ,  10.2    ],
       [110.239  ,  37.     ,   9.51   ,  14.87   ],
       [110.     ,  29.74   ,   9.51   ,  14.87   ],
       [ 46.052  ,  14.52   ,   7.27   ,   7.26   ],
       [ 70.367  ,  20.52   ,   8.55   ,  10.2    ],
       [ 70.367  ,  20.52   ,   8.55   ,  10.2    ],
       [ 86.     ,  21.24   ,   9.63   ,  10.62   ],
       [110.     ,  29.74   ,   9.51   ,  14.87   ],
       [ 88.5    ,  21.24   ,   9.63   ,  10.62   ],
       [ 70.367  ,  20.52   ,   8.55   ,  10.2    ],
       [ 88.5    ,  21.24   ,   9.63   ,  11.6

In [27]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.4, random_state=0)

# 6. Building a muilt-regression model

Our machine learning regression model for predicting a ship's "crew" size can be expressed as:

$$ \hat{y}_{i} = w_0 + \sum_{j=1}^{4} X_{ij} w_j $$

In [28]:
from sklearn.linear_model import LinearRegression
slr = LinearRegression()

regressor=slr.fit(X_train, y_train)
y_test_pred = regressor.predict(X_test)

# Evaluation of regression model

In [29]:
from sklearn.metrics import r2_score

print("The r2 for test set:",r2_score(y_test,y_test_pred))



The r2 for test set: 0.9088777678028096


# Regression coefficients

In [30]:
regressor.intercept_

-1.001258929743961

In [31]:
regressor.coef_

array([ 0.01875221, -0.14002277,  0.43403626,  0.73092915])

 Regularized Regression: Lasso

In [35]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Assuming you have already defined X and y

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

# Create the pipeline with StandardScaler and Ridge regression
alpha = 0.05
ridgeReg = Ridge(alpha=alpha)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', ridgeReg)
])

# Training the model using the pipeline
pipeline.fit(X_train, y_train)

# Predicting on the test set
pred = pipeline.predict(X_test)

# Calculating MSE and R2 score
mse = mean_squared_error(y_test, pred)
print("Mean Squared Error:", mse)

r2 = r2_score(y_test, pred)
print("The R2 for the test set:", r2)


Mean Squared Error: 0.9977356318169944
The R2 for the test set: 0.9089106726274393


In [37]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
# Assuming you have already defined X and y

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

# Create the pipeline with StandardScaler and Lasso regression
alpha = 0.05
lassoReg = Lasso(alpha=alpha)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lasso', lassoReg)
])

# Training the model using the pipeline
pipeline.fit(X_train, y_train)

# Predicting on the test set
las_pred = pipeline.predict(X_test)

# Calculating MSE and R2 score
mse = mean_squared_error(y_test, las_pred)
print("Mean Squared Error:", mse)

r2 = r2_score(y_test, las_pred)
print("The R2 for the test set:", r2)

Mean Squared Error: 1.0937710285636084
The R2 for the test set: 0.9001430197395943


In [38]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

# Assuming you have already defined X and y

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

# Create the ElasticNet model
alpha = 1
l1_ratio = 0.5
ENreg = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)

# Training the model
ENreg.fit(X_train, y_train)

# Predicting on the test set
pred_cv = ENreg.predict(X_test)

# Calculating MSE and R2 score
mse = mean_squared_error(y_test, pred_cv)
print("Mean Squared Error:", mse)

r2 = r2_score(y_test, pred_cv)
print("The R2 for the test set:", r2)


Mean Squared Error: 1.5822552092455742
The R2 for the test set: 0.8555463409887064
